In [1]:
#from google.colab import drive
#drive.mount('/content/Drive')

In [1]:
import pandas as pd
# df = pd.read_csv("/content/Drive/MyDrive/NLP/for_train.txt", sep="\t", header=None, names=["text", "rephrase"])
# print(df.head())

df = pd.read_csv("./output.txt", sep="|", header=None, names=["text", "rephrase"])
print(df.head())

                                                text  \
0  I need to get away from this city for a bit. I...   
1  Hate when people try to guilt trip you into so...   
2  Why does it have to be so hard to network if y...   
3  Children with autism use stereotypic behavior ...   
4  I have less than 2 hours to get home and make ...   

                                            rephrase  
0  I'm thankful for everything this city gave me!...  
1  I don’t like it when people try to guilt -trip...  
2  I'm glad networking with gaming comes very eas...  
3  Children with autism tend to demonstrate stere...  
4  I have less than 2 hours to get home and make ...  


In [3]:
# !pip install sentence-transformers

In [4]:
# !pip install datasets

In [2]:
import argparse
import pandas as pd
import numpy as np
import random
#from sentence_transformers import SentenceTransformer, util
import os
from transformers import Trainer, pipeline, set_seed, AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import nltk
nltk.download('punkt')
import csv
from datasets import load_dataset, load_metric, DatasetDict, Dataset


/Users/snehadarshini/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/snehadarshini/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
!pip3 install rouge_score

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [7]:
!pip3 install sacrebleu

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [8]:
metric = load_metric("rouge")
metric2 = load_metric("sacrebleu")
from transformers import BartTokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")

/var/folders/t3/0fprr8zn0vs0pk4bn0xd0r780000gn/T/ipykernel_34705/96984222.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [3]:
from sklearn.model_selection import train_test_split

# Assuming your dataframe is named 'df'
# Specify the test size as 0.05 to get a 95%/5% train/test split
train_df, test_df = train_test_split(df, test_size=0.10, random_state=42)

In [10]:
train_df.shape

(6011, 2)

In [4]:
train_data = train_df.iloc[0:4000]
val_data = train_df.iloc[4000:6011]

In [12]:
test_df.shape

(668, 2)

In [5]:
# convert to dataset
raw_data = DatasetDict({
    "train": Dataset.from_pandas(train_data),
    "eval": Dataset.from_pandas(val_data),
    "test": Dataset.from_pandas(test_df)
})

In [14]:
#train_df['original_with_label'] = train_df['text'].apply(lambda x: f"{x}: Strategy:[neutralizing]")

In [15]:
#test_df['original_with_label'] = test_df['text'].apply(lambda x: f"{x}: Strategy:[neutralizing]")

In [16]:
def preprocess_function(row):
    inputs = row["text"]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')
   # model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)
    # You can adjust the following line based on the column name containing the reframed text
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(row["rephrase"], max_length=512, truncation=True, padding='max_length')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [17]:
raw_data_encoded = raw_data.map(preprocess_function, batched=True)


Map:   0%|                                      | 0/4000 [00:00<?, ? examples/s]/Users/snehadarshini/Library/Python/3.9/lib/python/site-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|███████████████████████████| 668/668 [00:00<00:00, 4352.16 examples/s]


In [18]:
raw_data_encoded

DatasetDict({
    train: Dataset({
        features: ['text', 'rephrase', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4000
    })
    eval: Dataset({
        features: ['text', 'rephrase', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2011
    })
    test: Dataset({
        features: ['text', 'rephrase', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 668
    })
})

In [19]:
# !pip install accelerate==0.20.1

In [20]:
# !pip install transformers -U
# !pip install accelerate -U

In [21]:
# !pip install transformers -U
# !pip install accelerate -U
# !pip install torch -U

In [22]:
import torch
from transformers import BartForConditionalGeneration, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")
batch_size = 2
logging_steps = len(raw_data["train"])
#device = torch.device("mps")
#model.to(device)
args = Seq2SeqTrainingArguments(
        "text-rephrase",
        evaluation_strategy = "epoch",
        learning_rate=0.001,
        disable_tqdm=False,
        logging_steps=logging_steps,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        log_level="error",
        weight_decay=0.01,
        num_train_epochs=2,
        predict_with_generate=True,
        fp16=False,
    )
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [23]:
def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        # Replace -100 in the labels as we can't decode them.
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        # Rouge expects a newline after each sentence
        decoded_preds_joined = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
        decoded_labels_joined = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

        result = metric.compute(predictions=decoded_preds_joined, references=decoded_labels_joined, use_stemmer=True)
        # Extract a few results
        #result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
        result = {key: value.mid.fmeasure for key, value in result.items()}

        # Add mean generated length
        prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
        result["gen_len"] = np.mean(prediction_lens)

        # Add bleu scores
        decoded_labels_expanded = [[x] for x in decoded_labels]
        result2 = metric2.compute(predictions=decoded_preds, references=decoded_labels_expanded)
        result['sacrebleu'] = round(result2["score"], 1)

        return {k: round(v, 4) for k, v in result.items()}

In [24]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=raw_data_encoded["train"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    eval_dataset=raw_data_encoded["eval"],
    compute_metrics=compute_metrics
)

In [25]:

trainer.train()

trainer.save_model("output/reframer-bart-large")

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
trainer.evaluate()

In [7]:

# Load trained model
model = AutoModelForSeq2SeqLM.from_pretrained("output/reframer")
tokenizer = AutoTokenizer.from_pretrained("output/reframer")

reframer = pipeline('summarization', model=model, tokenizer=tokenizer)




In [ ]:
texts = raw_data['test']['text']
reframed_phrases = [reframer(phrase)[0]['summary_text'] for phrase in texts[:20]]

with open(os.path.join('output/reframer-bart-large/bart_5_epoch.txt'), 'w') as f:
        for item in reframed_phrases:
            f.write("%s\n" % item)

In [8]:
reframed_phrases = reframer("I do not like this product")

Your max_length is set to 128, but your input_length is only 8. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=4)


In [9]:
reframed_phrases

[{'summary_text': "IItEvenMyTheThisWhenThereSoSometimesIf'mGettingImPeopleCanTimeBeingWeAGoingHaving amAfterTodayAt hopeJustT haveLifeTheseAlthough needWorkingSome thoughCFeAllOnly"}]